In [43]:
import pandas as pd
import zstandard as zstd
import json

In [41]:
paths = 'data/UCSD_submissions.zst'

In [164]:
data = []
with open("data/UCSD_submissions.zst", 'rb') as fh:
    dctx = zstd.ZstdDecompressor(max_window_size=2147483648)
    with dctx.stream_reader(fh) as reader:
        previous_line = ""
        while True:
            chunk = reader.read(2**24)  # 16mb chunks
            if not chunk:
                break

            string_data = chunk.decode('utf-8')
            lines = string_data.split("\n")
            for i, line in enumerate(lines[:-1]):
                if i == 0:
                    line = previous_line + line
                object = json.loads(line)
                data.append(object)
                # do something with the object here
            previous_line = lines[-1]
with open("data/UCSD_comments.zst", 'rb') as fh:
    dctx = zstd.ZstdDecompressor(max_window_size=2147483648)
    with dctx.stream_reader(fh) as reader:
        previous_line = ""
        while True:
            chunk = reader.read(2**24)  # 16mb chunks
            if not chunk:
                break

            string_data = chunk.decode('utf-8')
            lines = string_data.split("\n")
            for i, line in enumerate(lines[:-1]):
                if i == 0:
                    line = previous_line + line
                object = json.loads(line)
                data.append(object)
                # do something with the object here
            previous_line = lines[-1]
df = pd.DataFrame(data)

In [202]:
df.head()

,archived,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,brand_safe,can_gild,...,quarantined,steward_reports,associated_award,collapsed_because_crowd_control,comment_type,collapsed_reason_code,author_is_blocked,unrepliable_reason,editable,created_date
0,True,spleeyah,None,,"[{'e': 'text', 't': 'Class of '12 | Q29tcHV0ZX...",Class of '12 | Q29tcHV0ZXIgU2NpZW5jZQo= (B.S.),None,richtext,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-10-13 01:37:56
1,True,spleeyah,None,,"[{'e': 'text', 't': 'Class of '12 | Q29tcHV0ZX...",Class of '12 | Q29tcHV0ZXIgU2NpZW5jZQo= (B.S.),None,richtext,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-10-13 06:36:29
2,True,[deleted],,None,NaN,None,dark,NaN,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-10-13 12:37:30
3,True,xxbondsxx,None,None,[],None,None,text,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-10-14 06:05:16
4,True,SnowdensOfYesteryear,None,None,[],None,None,text,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-10-15 08:02:04


In [181]:
# see all the columns
for i in range(0, len(df.columns), 10):
    print(df.columns[i:i+10])

Index(['archived', 'author', 'author_flair_background_color',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'brand_safe',
       'can_gild'],
      dtype='object')
Index(['contest_mode', 'created_utc', 'distinguished', 'domain', 'edited',
       'gilded', 'hidden', 'hide_score', 'id', 'is_crosspostable'],
      dtype='object')
Index(['is_reddit_media_domain', 'is_self', 'is_video', 'link_flair_css_class',
       'link_flair_richtext', 'link_flair_text', 'link_flair_text_color',
       'link_flair_type', 'locked', 'media'],
      dtype='object')
Index(['media_embed', 'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'retrieved_on', 'rte_mode',
       'score'],
      dtype='object')
Index(['secure_media', 'secure_media_embed', 'selftext', 'send_replies',
       'spoiler', 'stickied', 'subreddit', 'subreddit_id',
       'subreddit_name_prefixed', '

In [200]:
df.shape

(814431, 147)

## Data Wrangling

To reduce the dataset and keep it relevant, I will only use the posts from beginning of 2017 to end of 2022.

In [189]:
dates = pd.to_datetime(df['created_utc'], unit = 's')

In [190]:
# to reduce the dataset, find entries that date from the beginning of 2017
df['created_date'] = dates
new_df = df[df['created_date'] >= '2017-1-01']

In [191]:
new_df.shape

(706829, 147)

In [193]:
# get necessary text from selftext (the content of the original submissions)
# and the bodies of comments of the submissions
new_df = new_df[['selftext','body']]

In [194]:
# we can see that one of the two, selftext and body columns, are nan as they only can have one or the other.
new_df.head()

,selftext,body
12183,Happy New Year UCSD! Hope everyone reflected o...,NaN
12184,Friday so you can have the weekend to chill? T...,NaN
12185,Since the S. Quarterly parking permit isn't av...,NaN
12186,I'm at a loss nowadays. I'm a third year NanoE...,NaN
12187,"Hey guys, \n\nI'm wondering what the best way ...",NaN


In [196]:
# set dataframe to contain both selftext and body columns into a combined column called text
text = new_df['selftext'].fillna(new_df['body'])
combined = pd.DataFrame(text)
combined.columns = ['text']
combined.head()

,text
12183,Happy New Year UCSD! Hope everyone reflected o...
12184,Friday so you can have the weekend to chill? T...
12185,Since the S. Quarterly parking permit isn't av...
12186,I'm at a loss nowadays. I'm a third year NanoE...
12187,"Hey guys, \n\nI'm wondering what the best way ..."


In [197]:
# remove all the '[removed]', '[deleted]', and empty entries for text
cond1 = ~(combined['text'] == '')
cond2 = ~(combined['text'] == '[removed]')
cond3 = ~(combined['text'] == '[deleted]')
combined = combined[cond1 & cond2 & cond3]
combined.shape

(624471, 1)

In [198]:
# save combined
combined.head()

,text
12183,Happy New Year UCSD! Hope everyone reflected o...
12184,Friday so you can have the weekend to chill? T...
12185,Since the S. Quarterly parking permit isn't av...
12186,I'm at a loss nowadays. I'm a third year NanoE...
12187,"Hey guys, \n\nI'm wondering what the best way ..."


In [199]:
combined.iloc[0:10]['text']

12183    Happy New Year UCSD! Hope everyone reflected o...
12184    Friday so you can have the weekend to chill? T...
12185    Since the S. Quarterly parking permit isn't av...
12186    I'm at a loss nowadays. I'm a third year NanoE...
12187    Hey guys, \n\nI'm wondering what the best way ...
12188    my friend is switching out of an impacted majo...
12189    If I ask for a grade change for last quarter n...
12190    I forgot my ID and room key at home so I need ...
12191    I'm a high school junior in the SoCal area int...
12192          Sick of dining hall food so help me out fam
Name: text, dtype: object

In [206]:
import torch
from collections import Counter
from torch.utils.data import Dataset, DataLoader

OSError: dlopen(/Users/nick/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_global_deps.dylib, 0x000A): Library not loaded: @rpath/libmkl_core.1.dylib
  Referenced from: <57A6455F-0260-326D-9143-DE9307A6AF48> /Users/nick/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_global_deps.dylib
  Reason: tried: '/Users/nick/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libmkl_core.1.dylib' (no such file), '/Users/nick/opt/anaconda3/lib/python3.9/site-packages/torch/lib/../../../../libmkl_core.1.dylib' (no such file), '/Users/nick/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libmkl_core.1.dylib' (no such file), '/Users/nick/opt/anaconda3/lib/python3.9/site-packages/torch/lib/../../../../libmkl_core.1.dylib' (no such file), '/Users/nick/opt/anaconda3/lib/python3.9/lib-dynload/../../libmkl_core.1.dylib' (no such file), '/Users/nick/opt/anaconda3/bin/../lib/libmkl_core.1.dylib' (no such file), '/usr/local/lib/libmkl_core.1.dylib' (no such file), '/usr/lib/libm